1.Veri kümesindeki notların ağırlıklı ortalamasını içeren bir değişken oluşturun. Dördüncü sınıftaki öğrencilerin sayısı ile sekizinci sınıftaki öğrencilerin sayısı farklı. Bu yüzden ağırlıklı ortalamaya ihtiyacınız olacak!

2.Yeni oluşturduğunuz değişken ile harcama çeşitlerinin korelasyonu nedir? Hangi harcama kaleminin korelasyonu diğerlerine göre fazladır?

3.Şimdi dört harcama kalemi için Temel Bileşenler Analizi (PCA) uygulayın! Toplam varyansın ne kadarı ilk bileşen tarafından açıklanabilmektedir?

4.Oluşturduğunuz genel not ortalaması ve ilk temel bileşen arasındaki korelasyon nedir?

5.Modeliniz için en uygun değişkenleri seçmeniz gerektiğinde, harcama kalemlerinin yerine ilk temel değişkenleri tercih eder miydiniz? Neden?



In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from matplotlib.mlab import PCA as mlabPCA
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")
baslik_font = {'family': 'arial', 'color': 'darkred','weight': 'bold','size': 13 }
eksen_font  = {'family': 'arial', 'color': 'darkblue','weight': 'bold','size': 10 }

In [20]:
states_all2 =pd.read_csv("states_all.csv")
states_all2.head()

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN


1.Veri kümesindeki notların ağırlıklı ortalamasını içeren bir değişken oluşturun. Dördüncü sınıftaki öğrencilerin sayısı ile sekizinci sınıftaki öğrencilerin sayısı farklı. Bu yüzden ağırlıklı ortalamaya ihtiyacınız olacak!

In [31]:
states_all2["overall_score"] = (states_all2["GRADES_4_G"]*((states_all2["AVG_MATH_4_SCORE"] +
states_all2["AVG_READING_4_SCORE"])/2) + states_all2["GRADES_8_G"] * ((states_all2["AVG_MATH_8_SCORE"] + states_all2["AVG_READING_8_SCORE"])/2))/(states_all2["GRADES_4_G"] + states_all2["GRADES_8_G"])

2.Yeni oluşturduğunuz değişken ile harcama çeşitlerinin korelasyonu nedir? Hangi harcama kaleminin korelasyonu diğerlerine göre fazladır? 

In [33]:
print(overall_score)

NameError: name 'overall_score' is not defined

In [9]:
states_all2[["TOTAL_EXPENDITURE","INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE","overall_score"]].corr()

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,overall_score
TOTAL_EXPENDITURE,1.000000,0.991347,0.993314,0.946110,0.089937
INSTRUCTION_EXPENDITURE,0.991347,1.000000,0.976889,0.909340,0.103678
SUPPORT_SERVICES_EXPENDITURE,0.993314,0.976889,1.000000,0.953091,0.094103
OTHER_EXPENDITURE,0.946110,0.909340,0.953091,1.000000,0.004678
overall_score,0.089937,0.103678,0.094103,0.004678,1.000000


INSTRUCTION_EXPENDITURE ile en yuksek korelasyon goruyoruz. Diger harcamalarla ise dusuk bir korelasyon gorulmektedir. 


In [29]:
#Korelasyon Matrisi 

A = states_all2[["TOTAL_EXPENDITURE","INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE","overall_score"]]

plt.figure(figsize=(20,20))
sns.heatmap(A, annot=True, linewidths=.1, vmin=-1, vmax=1, cmap='viridis')
plt.title("Korelasyon Matrisi (state_all3)", fontdict = baslik_font)

KeyError: "['overall_score'] not in index"

3.Şimdi dört harcama kalemi için Temel Bileşenler Analizi (PCA) uygulayın! Toplam varyansın ne kadarı ilk bileşen tarafından açıklanabilmektedir?


In [ ]:
# BU SORUDA ASAGIDAKI GIBI OLAN STATES3 ILE DENEDIM OLMADI SONRA states_all_df OLUSTURUP DENEDIM'PRIMARY KEY 
#DEKI BIR DEGERIN FLOAT OLMASINDAN OTURU OLMADI
# 29. SATIRDAKI GIBI DENEDIM BU DA OLMADI

In [10]:
states_all3 = states_all2[["TOTAL_EXPENDITURE", "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE", "OTHER_EXPENDITURE"]]
states_all3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 4 columns):
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
dtypes: float64(4)
memory usage: 46.7 KB


In [ ]:
sklearn_pca = PCA(n_components=5)
Y_sklearn = sklearn_pca.fit_transform(X)

P = eig_vec_cov[:, 0] 

Y = P.T.dot(Xt) 

print(
    'Veri kümesindeki toplam varyans yüzdesi',
    'Elle hesaplanan bileşen.\n',
    sklearn_pca.explained_variance_ratio_
)

# Compare the sklearn solution to ours – a perfect match.
plt.plot(Y_sklearn[:, 0], Y, 'o')
plt.title('Çözümlerin Karşılaştırılması')
plt.ylabel('Sklearn Bileşeni 1')
plt.xlabel('Elle Hesaplanan Bileşen 1')
plt.show()

In [15]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


A = states_all3
print(A)
# create the PCA instance
pca = PCA(2)
# fit on data
pca.fit(A)
# access values and vectors
print(pca.components_)
print(pca.explained_variance_)
# transform data
B = pca.transform(A)
print(B)

      TOTAL_EXPENDITURE  INSTRUCTION_EXPENDITURE  \
0             2653798.0                1481703.0   
1              972488.0                 498362.0   
2             3401580.0                1435908.0   
3             1743022.0                 964323.0   
4            27138832.0               14358922.0   
5             3264826.0                1642466.0   
6             3721338.0                2148041.0   
7              638784.0                 372722.0   
8              742893.0                 329160.0   
9            11305642.0                5166374.0   
10            5535942.0                3043984.0   
11            1040121.0                 536115.0   
12             886161.0                 473505.0   
13            9850560.0                5010400.0   
14            5182754.0                2598925.0   
15            2795774.0                1446478.0   
16            2234915.0                1165749.0   
17            2518082.0                1343438.0   
18          

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [16]:
sklearn_pca = PCA(n_components=5)

X = StandardScaler().fit_transform(A)

Xt = X.T
Cx = np.cov(Xt)
print('Kovaryans matrisi :\n', Cx)

Kovaryans matrisi :
 [[nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]
 [nan nan nan nan]]


4.Oluşturduğunuz genel not ortalaması ve ilk temel bileşen arasındaki korelasyon nedir?

In [24]:
import seaborn as sns
plt.figure(figsize=(20,20))
sns.heatmap(states_all2, annot=True, linewidths=.1, vmin=-1, vmax=1, cmap='viridis')
plt.title("Korelasyon Matrisi (state_all3)", fontdict = baslik_font)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

<Figure size 1440x1440 with 0 Axes>